In [58]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [59]:
# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [60]:
# import our chat-bot intents file
import json
with open('data/responses.json') as json_data:
    intents = json.load(json_data)

In [61]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

8 documents
2 classes ['goodbye', 'greeting']
14 unique stemmed words ['anyon', 'ar', 'bye', 'day', 'good', 'goodby', 'hello', 'hi', 'how', 'is', 'lat', 'see', 'ther', 'you']


In [62]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-62-9d149b3413ff>:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [63]:
# reset underlying graph data
tf.compat.v1.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('models/model.tflearn')

---------------------------------
Run id: IM3Z05
Log directory: tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 8
Validation samples: 0
--
Training Step: 1  | time: 0.105s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 8/8
--
Training Step: 2  | total loss: 0.62383 | time: 0.004s
| Adam | epoch: 002 | loss: 0.62383 - acc: 0.3375 -- iter: 8/8
--
Training Step: 3  | total loss: 0.68030 | time: 0.005s
| Adam | epoch: 003 | loss: 0.68030 - acc: 0.5727 -- iter: 8/8
--
Training Step: 4  | total loss: 0.68949 | time: 0.004s
| Adam | epoch: 004 | loss: 0.68949 - acc: 0.6119 -- iter: 8/8
--
Training Step: 5  | total loss: 0.69141 | time: 0.004s
| Adam | epoch: 005 | loss: 0.69141 - acc: 0.6210 -- iter: 8/8
--
Training Step: 6  | total loss: 0.69176 | time: 0.003s
| Adam | epoch: 006 | loss: 0.69176 - acc: 0.6236 -- iter: 8/8
--
Training Step: 7  | total loss: 0.69169 | time: 0.0

In [64]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )